<a href="https://colab.research.google.com/github/sh-0620/dacon-recommender-system/blob/main/stopwords_tf_idf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_path = '/content/drive/MyDrive/데이콘/기사추천시스템/open/'

In [4]:
article_info = pd.read_csv(data_path + 'article_info.csv')

In [5]:
article_info #기사 정보(userid = 기사 작성자)

,articleID,Title,Content,Format,Language,userID,userCountry,userRegion
0,ARTICLE_0000,19 Tips For Everyday Git Use,I've been using git full time for the past 4 y...,HTML,en,USER_0683,NaN,NaN
1,ARTICLE_0001,Intel buys computer vision startup Itseez to i...,Intel has acquired computer vision and machine...,HTML,en,USER_1129,NaN,NaN
2,ARTICLE_0002,Practical End-to-End Testing with Protractor,One of the reasons AngularJS is so great to wo...,HTML,en,USER_0256,NaN,NaN
3,ARTICLE_0003,Corporate venture growth in Brazil is another ...,Despite recent positive news and a renewed int...,HTML,en,USER_1304,NaN,NaN
4,ARTICLE_0004,Cross-channel user experiences with Drupal (aw...,"Last year around this time, I wrote that The B...",HTML,en,USER_0336,NaN,NaN
...,...,...,...,...,...,...,...,...
3003,ARTICLE_3003,Como consumir conteúdo de qualidade em iOS - C...,"Quando iniciei minha jornada em Swift, saindo ...",HTML,pt,USER_0882,BR,MG
3004,ARTICLE_3004,Aurelia 1.0 is Here!!!,It's been an amazing journey to get here and i...,HTML,en,USER_0220,NaN,NaN
3005,ARTICLE_3005,Lessons from converting an app to 100% Kotlin ...,This is part one in a series of posts about Ko...,HTML,en,USER_1010,BR,SP
3006,ARTICLE_3006,ITA está oferecendo 10 cursos gratuitos a dist...,"O Instituto Tecnológico de Aeronáutica (ITA) ,...",HTML,pt,USER_1210,NaN,NaN


In [6]:
article_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3008 entries, 0 to 3007
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   articleID    3008 non-null   object
 1   Title        3008 non-null   object
 2   Content      3008 non-null   object
 3   Format       3008 non-null   object
 4   Language     3008 non-null   object
 5   userID       3008 non-null   object
 6   userCountry  659 non-null    object
 7   userRegion   657 non-null    object
dtypes: object(8)
memory usage: 188.1+ KB


In [7]:
article_info

,articleID,Title,Content,Format,Language,userID,userCountry,userRegion
0,ARTICLE_0000,19 Tips For Everyday Git Use,I've been using git full time for the past 4 y...,HTML,en,USER_0683,NaN,NaN
1,ARTICLE_0001,Intel buys computer vision startup Itseez to i...,Intel has acquired computer vision and machine...,HTML,en,USER_1129,NaN,NaN
2,ARTICLE_0002,Practical End-to-End Testing with Protractor,One of the reasons AngularJS is so great to wo...,HTML,en,USER_0256,NaN,NaN
3,ARTICLE_0003,Corporate venture growth in Brazil is another ...,Despite recent positive news and a renewed int...,HTML,en,USER_1304,NaN,NaN
4,ARTICLE_0004,Cross-channel user experiences with Drupal (aw...,"Last year around this time, I wrote that The B...",HTML,en,USER_0336,NaN,NaN
...,...,...,...,...,...,...,...,...
3003,ARTICLE_3003,Como consumir conteúdo de qualidade em iOS - C...,"Quando iniciei minha jornada em Swift, saindo ...",HTML,pt,USER_0882,BR,MG
3004,ARTICLE_3004,Aurelia 1.0 is Here!!!,It's been an amazing journey to get here and i...,HTML,en,USER_0220,NaN,NaN
3005,ARTICLE_3005,Lessons from converting an app to 100% Kotlin ...,This is part one in a series of posts about Ko...,HTML,en,USER_1010,BR,SP
3006,ARTICLE_3006,ITA está oferecendo 10 cursos gratuitos a dist...,"O Instituto Tecnológico de Aeronáutica (ITA) ,...",HTML,pt,USER_1210,NaN,NaN


In [8]:
view_log = pd.read_csv(data_path + 'view_log.csv')

In [9]:
view_log #사용자-기사 로그데이터

,userID,articleID,userRegion,userCountry
0,USER_0000,ARTICLE_0661,NY,US
1,USER_0000,ARTICLE_2316,NY,US
2,USER_0000,ARTICLE_1345,NY,US
3,USER_0000,ARTICLE_1089,NY,US
4,USER_0000,ARTICLE_1484,NY,US
...,...,...,...,...
42712,USER_1420,ARTICLE_0682,SP,BR
42713,USER_1420,ARTICLE_2179,SP,BR
42714,USER_1420,ARTICLE_1848,SP,BR
42715,USER_1420,ARTICLE_0030,SP,BR


In [10]:
view_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42717 entries, 0 to 42716
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   userID       42717 non-null  object
 1   articleID    42717 non-null  object
 2   userRegion   42717 non-null  object
 3   userCountry  42717 non-null  object
dtypes: object(4)
memory usage: 1.3+ MB


# remove stopwords

In [11]:
import nltk
from nltk.corpus import stopwords
import re

# nltk 불용어 다운로드
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words_english = set(stopwords.words('english')) #영어
stop_words_portuguese = set(stopwords.words('portuguese')) #포르투갈어

combined_stop_words = stop_words_english.union(stop_words_portuguese)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [12]:
add_stopword = {'one','two','1','2','also','would','could'}

# 추가 불용어를 combined_stop_words에 추가
for word in add_stopword:
    combined_stop_words.add(word)

In [13]:
from nltk.stem import WordNetLemmatizer

# 표제어 처리 객체 생성
lemmatizer = WordNetLemmatizer()

# 텍스트 전처리 함수
def preprocess(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    words = text.split()
    words = [word for word in words if word not in combined_stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]
    words = [word for word in words if len(word) >= 2]

    return words

In [14]:
article_info['Title'] = article_info['Title'].apply(preprocess)
# article_info['Content'] = article_info['Content'].apply(preprocess)

In [15]:
# 각 행의 리스트를 문자열로 변환하는 함수 정의
def list_to_string(row):
    return ' '.join(row)

# 각 행의 리스트를 문자열로 변환
article_info['Title'] = article_info['Title'].apply(list_to_string)
# article_info['Content'] = article_info['Content'].apply(list_to_string)


# TF-IDF

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

In [17]:
# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(max_features=5000) #stop_words='english'는 영어 불용어를 제거하고, max_features=5000는 최대 5000개의 단어 특성을 사용한다는 의미입니다.
tfidf_matrix = tfidf_vectorizer.fit_transform(article_info['Title'])
# tfidf_matrix = tfidf_vectorizer.fit_transform(article_info['Title'] + ' ' + article_info['Content'])
# 코사인 유사도 계산
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [18]:
def get_personalized_recommendations(user_id, cosine_sim=cosine_sim, articles_df=article_info):
    # 사용자가 읽은 기사 목록 가져오기
    user_articles = view_log[view_log['userID'] == user_id]['articleID'].tolist()

    # 사용자가 읽은 기사의 인덱스 가져오기
    article_indices = article_info[article_info['articleID'].isin(user_articles)].index.tolist()

    # 유사도 점수 합산
    sim_scores = cosine_sim[article_indices].sum(axis=0)

    # 유사도 점수를 기준으로 기사 정렬
    ranked_indices = sim_scores.argsort()[::-1]

    # 상위 5개 기사 추천
    recommended_article_indices = ranked_indices[:5]

    return articles_df.iloc[recommended_article_indices]



In [19]:
recommendations = []

for idx, user in enumerate(view_log.userID.unique()):
    recommendations.append([user, get_personalized_recommendations(user).articleID.values])

recommendations = pd.DataFrame(recommendations, columns=['userID', 'articleID'])

recommendations = recommendations.explode('articleID').reset_index(drop=True)


In [20]:
recommendations

,userID,articleID
0,USER_0000,ARTICLE_0884
1,USER_0000,ARTICLE_2757
2,USER_0000,ARTICLE_1260
3,USER_0000,ARTICLE_1230
4,USER_0000,ARTICLE_1033
...,...,...
7070,USER_1420,ARTICLE_1507
7071,USER_1420,ARTICLE_2726
7072,USER_1420,ARTICLE_1901
7073,USER_1420,ARTICLE_1732


In [21]:
recommendations.to_csv(data_path + 'user_except_title.csv')  # 제거해도 성능이 높아지지 않음

## 많이 등장하는 단어 확인

In [22]:
from collections import Counter

# 모든 텍스트 데이터 결합 (Title과 Content 결합)
all_text = ' '.join(article_info['Title'] + ' ' + article_info['Content'])

# 전처리된 단어 리스트 생성
processed_words = preprocess(all_text)

# 단어 빈도 계산
# word_counter = Counter(processed_words)
lem_word_counter = Counter(processed_words)

# 빈도수에 따라 단어 정렬
# sorted_word_counts = word_counter.most_common()
lem_sorted_word_counts = lem_word_counter.most_common()

# 결과 출력
print(lem_sorted_word_counts[:50])  # 상위 50개 단어 출력

[('data', 6594), ('new', 6085), ('time', 6023), ('google', 5936), ('like', 5422), ('user', 5025), ('use', 4989), ('company', 4826), ('make', 4248), ('need', 4149), ('get', 3916), ('work', 3885), ('service', 3874), ('way', 3871), ('people', 3639), ('system', 3364), ('team', 3257), ('using', 3248), ('year', 3180), ('product', 3150), ('code', 3123), ('see', 3114), ('customer', 3095), ('learning', 3006), ('app', 2968), ('thing', 2966), ('first', 2851), ('many', 2847), ('example', 2835), ('business', 2811), ('machine', 2748), ('want', 2710), ('even', 2654), ('technology', 2541), ('change', 2498), ('platform', 2497), ('cloud', 2492), ('software', 2460), ('application', 2443), ('say', 2420), ('feature', 2382), ('take', 2315), ('well', 2296), ('developer', 2282), ('different', 2279), ('help', 2208), ('project', 2184), ('used', 2168), ('much', 2160), ('digital', 2148)]


In [23]:
lem_sorted_word_counts[:50]

[('data', 6594),
 ('new', 6085),
 ('time', 6023),
 ('google', 5936),
 ('like', 5422),
 ('user', 5025),
 ('use', 4989),
 ('company', 4826),
 ('make', 4248),
 ('need', 4149),
 ('get', 3916),
 ('work', 3885),
 ('service', 3874),
 ('way', 3871),
 ('people', 3639),
 ('system', 3364),
 ('team', 3257),
 ('using', 3248),
 ('year', 3180),
 ('product', 3150),
 ('code', 3123),
 ('see', 3114),
 ('customer', 3095),
 ('learning', 3006),
 ('app', 2968),
 ('thing', 2966),
 ('first', 2851),
 ('many', 2847),
 ('example', 2835),
 ('business', 2811),
 ('machine', 2748),
 ('want', 2710),
 ('even', 2654),
 ('technology', 2541),
 ('change', 2498),
 ('platform', 2497),
 ('cloud', 2492),
 ('software', 2460),
 ('application', 2443),
 ('say', 2420),
 ('feature', 2382),
 ('take', 2315),
 ('well', 2296),
 ('developer', 2282),
 ('different', 2279),
 ('help', 2208),
 ('project', 2184),
 ('used', 2168),
 ('much', 2160),
 ('digital', 2148)]